In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [69]:
conn = sq.connect('../../../../Data-Pipelines/Outputs/CensusBureau.db')
#ACS52017
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2017_CEDS]', conn)
ACS52017 = pd.DataFrame(sql_query)
twothousandseventeen = ACS52017.set_index(['NAME', 'GEO_ID'])
twothousandseventeen = twothousandseventeen.add_suffix(' 2017')
twothousandseventeen = twothousandseventeen.reset_index(drop = False)
#ACS52022
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_1]', conn)
ACS52022_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_2]', conn)
ACS52022_2 = pd.DataFrame(sql_query)
ACS52022 = ACS52022_1.merge(ACS52022_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52022.drop(columns = 'Source', inplace = True)
twothousandtwentytwo = ACS52022.set_index(['NAME', 'GEO_ID'])
twothousandtwentytwo = twothousandtwentytwo.add_suffix(' 2022')
twothousandtwentytwo = twothousandtwentytwo.reset_index(drop = False)

In [70]:
twothousandseventeen.head()

,NAME,GEO_ID,Internet:Has a Computer 2017,Internet%:Has a Computer 2017,Internet:Has a Computer Under 18 2017,Internet%:Under 18 has a Computer 2017,Internet:Has a Computer 18 to 64 2017,Internet%:18 to 64 has a Computer 2017,Internet:Has a Computer 65 Over 2017,Internet%:65 Over has a Computer 2017,Internet:Has a Computer with Dial Up Only 2017,Internet%:Has a Computer with Dial Up Only 2017,Internet:Has a Computer with Dial Up Only Under 18 2017,Internet%:Under 18 has a Computer with Dial Up Only 2017,Internet:Has a Computer with Dial Up Only 18 to 64 2017,Internet%:18 to 64 has a Computer with Dial Up Only 2017,Internet:Has a Computer with Dial Up Only 65 Over 2017,Internet%:65 Over has a Computer with Dial Up Only 2017,Internet:Has a Computer with Broadband 2017,Internet%:Has a Computer with Broadband 2017,Internet:Has a Computer with Broadband Under 18 2017,Internet%:Under 18 has a Computer with Broadband 2017,Internet:Has a Computer with Broadband 18 to 64 2017,Internet%:18 to 64 has a Computer with Broadband 2017,Internet:Has a Computer with Broadband 65 Over 2017,Internet%:65 Over has a Computer with Broadband 2017,Internet:Has a Computer with No Internet 2017,Internet%:Has a Computer with No Internet 2017,Internet:Has a Computer with No Internet Under 18 2017,Internet%:Under 18 has a Computer with No Internet 2017,Internet:Has a Computer with No Internet 18 to 64 2017,Internet%:18 to 64 has a Computer with No Internet 2017,Internet:Has a Computer with No Internet 65 Over 2017,Internet%:65 Over has a Computer with No Internet 2017,Internet:Has No Computer 2017,Internet%:Has No Computer 2017,Internet:Has No Computer Under 18 2017,Internet%:Under 18 has No Computer 2017,Internet:Has No Computer 18 to 64 2017,Internet%:18 to 64 has No Computer 2017,Internet:Has No Computer 65 Over 2017,Internet%:65 Over has No Computer 2017,Ed:Population 25+ Educational Attainment 2017,Ed:Less than High School 2017,Ed%:Less than High School 2017,Ed:High School Graduate or Equivalency 2017,Ed%:High School Graduate or Equivalency 2017,Ed:High School Graduate or More 2017,Ed%:High School Graduate or More 2017,Ed:Some College 2017,Ed%:Some College 2017,Ed%:Some College or More 2017,Ed:Associates 2017,Ed%:Associates 2017,Ed:Bachelors 2017,Ed%:Bachelors 2017,Ed:Bachelors or More 2017,Ed%:Bachelors or More 2017,Ed:Masters 2017,Ed:Masters or More 2017,Ed%:Masters or More 2017,Ed%:Masters 2017,Ed:Professional School Degree 2017,Ed%:Professional School Degree 2017,Ed%:Professional School Degree or More 2017,Ed:Doctorate Degree 2017,Ed%:Doctorate Degree 2017,Employment:Armed Forces 2017,Employment:Not in Labor Force 2017,Employment:Employed Civilian 2017,Employment:Unemployed Civilian 2017,Employment%:Civilian Population in LF 16 Over that are Employed 2017,Employment%:Total Civilian Population 16 Over that are Employed 2017,Employment%:Population 16 Over in LF that are Employed 2017,Employment%:Population 16 Over that are Employed Civilians 2017,Employment%:Civilian Population in LF 16 Over that are Unemployed 2017,Employment%:Total Civilian Population 16 Over that are Unemployed 2017,Employment%:Population 16 Over in LF that are Unemployed 2017,Employment%:Population 16 Over that are Unemployed Civilians 2017,Employment%:Civilian Population 16 Over Not in Labor Force 2017,Employment%:Total Population 16 Over Not in Labor Force 2017,Employment:Total Population 16 Over In Labor Force 2017,Employment%:Total Population 16 Over In Labor Force 2017,Employment:Civilian Population 16 Over In Labor Force 2017,Employment%:Civilian Population 16 Over In Labor Force 2017,Employment%:Population 16 Over in LF Employed in Armed Forces 2017,Employment%:Population 16 Over Employed in Armed Forces 2017,HHIncome:Median for Householder Under 25 2017,HHIncome:Median for Householder Under 25 to 44 2017,HHIncome:Median for Householder Under 45 to 64 2017,HHIncome:Median for Householder Under 65 Over 2017,Median Household Income 2017,HHIncome:Total Households 2017,"HHInc

In [71]:
#geographies
geos = ['Cheatham County', 'Davidson County', 'Dickson County', 
        'Robertson County', 'Rutherford County', 'Sumner County', 'Trousdale County', 'Williamson County', 
        'Wilson County', 'Montgomery County', 'Stewart County', 'Houston County', 'Humphreys County', 'GNRC', 'Tennessee', 'US'
       ]

In [72]:
dfs = [twothousandseventeen, twothousandtwentytwo]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'], how='outer'), dfs)

In [73]:
dfsuff = dfsuff.reset_index().set_index('NAME').transpose()
dfsuff = dfsuff.rename(columns = namestocommon)
dfsuff = dfsuff[geos]
dfsuff = dfsuff.transpose().reset_index()

In [74]:
cols = list(dfsuff.columns)
cols.remove('NAME')
cols.remove('GEO_ID')
dfsuff[cols] = dfsuff[cols].astype(float)

In [75]:
data = dfsuff

In [76]:
cols = ['NAME', 
#         'Internet:Has a Computer 2017', 'Internet%:Has a Computer 2017', 'Internet:Has a Computer Under 18 2017', 'Internet%:Under 18 has a Computer 2017', 
#         'Internet:Has a Computer 18 to 64 2017', 'Internet%:18 to 64 has a Computer 2017', 'Internet:Has a Computer 65 Over 2017', 
#         'Internet%:65 Over has a Computer 2017', 'Internet:Has a Computer with Dial Up Only 2017', 'Internet%:Has a Computer with Dial Up Only 2017', 
#         'Internet:Has a Computer with Dial Up Only Under 18 2017', 'Internet%:Under 18 has a Computer with Dial Up Only 2017', 
#         'Internet:Has a Computer with Dial Up Only 18 to 64 2017', 'Internet%:18 to 64 has a Computer with Dial Up Only 2017', 
#         'Internet:Has a Computer with Dial Up Only 65 Over 2017', 'Internet%:65 Over has a Computer with Dial Up Only 2017', 
        'Internet:Has a Computer with Broadband 2017', 'Internet%:Has a Computer with Broadband 2017', 
#         'Internet:Has a Computer with Broadband Under 18 2017', 
#         'Internet%:Under 18 has a Computer with Broadband 2017', 'Internet:Has a Computer with Broadband 18 to 64 2017', 
#         'Internet%:18 to 64 has a Computer with Broadband 2017', 'Internet:Has a Computer with Broadband 65 Over 2017', 
#         'Internet%:65 Over has a Computer with Broadband 2017', 'Internet:Has a Computer with No Internet 2017', 'Internet%:Has a Computer with No Internet 2017', 
#         'Internet:Has a Computer with No Internet Under 18 2017', 'Internet%:Under 18 has a Computer with No Internet 2017', 
#         'Internet:Has a Computer with No Internet 18 to 64 2017', 'Internet%:18 to 64 has a Computer with No Internet 2017', 
#         'Internet:Has a Computer with No Internet 65 Over 2017', 'Internet%:65 Over has a Computer with No Internet 2017', 'Internet:Has No Computer 2017', 
#         'Internet%:Has No Computer 2017', 'Internet:Has No Computer Under 18 2017', 'Internet%:Under 18 has No Computer 2017', 
#         'Internet:Has No Computer 18 to 64 2017', 'Internet%:18 to 64 has No Computer 2017', 'Internet:Has No Computer 65 Over 2017', 
#         'Internet%:65 Over has No Computer 2017',
#         'Ed:Population 25+ Educational Attainment 2017', 'Ed:Less than High School 2017', 'Ed%:Less than High School 2017', 
#         'Ed:High School Graduate or Equivalency 2017', 'Ed%:High School Graduate or Equivalency 2017', 
        'Ed:High School Graduate or More 2017', 'Ed%:High School Graduate or More 2017', 
#         'Ed:Some College 2017', 'Ed%:Some College 2017', 'Ed%:Some College or More 2017', 'Ed:Associates 2017', 
#         'Ed%:Associates 2017', 'Ed:Bachelors 2017', 'Ed%:Bachelors 2017', 
        'Ed:Bachelors or More 2017', 'Ed%:Bachelors or More 2017', 
#         'Ed:Masters 2017', 'Ed:Masters or More 2017', 'Ed%:Masters or More 2017', 'Ed%:Masters 2017', 'Ed:Professional School Degree 2017', 
#         'Ed%:Professional School Degree 2017', 'Ed%:Professional School Degree or More 2017', 'Ed:Doctorate Degree 2017', 'Ed%:Doctorate Degree 2017', 
#         'Employment:Armed Forces 2017', 
#         'Employment:Not in Labor Force 2017', 'Employment:Employed Civilian 2017', 'Employment:Unemployed Civilian 2017', 
#         'Employment%:Civilian Population in LF 16 Over that are Employed 2017', 'Employment%:Total Civilian Population 16 Over that are Employed 2017', 
#         'Employment%:Population 16 Over in LF that are Employed 2017', 'Employment%:Population 16 Over that are Employed Civilians 2017', 
#         'Employment%:Civilian Population in LF 16 Over that are Unemployed 2017', 'Employment%:Total Civilian Population 16 Over that are Unemployed 2017', 
#         'Employment%:Population 16 Over in LF that are Unemployed 2017', 'Employment%:Population 16 Over that are Unemployed Civilians 2017', 
#         'Employment%:Civilian Population 16 Over Not in Labor Force 2017', 'Employment%:Total Population 16 Over Not in Labor Force 2017', 
#         'Employment:Total Population 16 Over In Labor Force 2017', 'Employment%:Total Population 16 Over In Labor Force 2017', 
        'Employment:Civilian Population 16 Over In Labor Force 2017', 'Employment%:Civilian Population 16 Over In Labor Force 2017', 
#         'Employment%:Population 16 Over in LF Employed in Armed Forces 2017', 'Employment%:Population 16 Over Employed in Armed Forces 2017', 
#         'HHIncome:Median for Householder Under 25 2017', 'HHIncome:Median for Householder Under 25 to 44 2017', 
#         'HHIncome:Median for Householder Under 45 to 64 2017', 'HHIncome:Median for Householder Under 65 Over 2017', 
        
        'Median Household Income 2017', 
        
#         'HHIncome:Total Households 2017', 'HHIncome:Less than 10,000 2017', 'HHIncome%:Less than 10,000 2017', 'HHIncome:10 to 14,999 2017', 
#         'HHIncome%:10 to 14,999 2017', 'HHIncome:15 to 19,999 2017', 'HHIncome%:15 to 19,999 2017', 'HHIncome:20 to 24,999 2017', 'HHIncome%:20 to 24,999 2017', 
#         'HHIncome:25 to 29,999 2017', 'HHIncome%:25 to 29,999 2017', 'HHIncome:30 to 34,999 2017', 'HHIncome:%30 to 34,999 2017', 'HHIncome:35 to 39,999 2017', 
#         'HHIncome%:35 to 39,999 2017', 'HHIncome:40 to 44,999 2017', 'HHIncome%:40 to 44,999 2017', 'HHIncome:45 to 49,999 2017', 'HHIncome%:45 to 49,999 2017', 
#         'HHIncome:50 to 59,999 2017', 'HHIncome%:50 to 59,999 2017', 'HHIncome:60 to 74,999 2017', 'HHIncome%:60 to 74,999 2017', 'HHIncome:75 to 99,999 2017', 
#         'HHIncome%:75 to 99,999 2017', 'HHIncome:100 to 124,999 2017', 'HHIncome%:100 to 124,999 2017', 'HHIncome:125 to 149,999 2017', 
#         'HHIncome%:125 to 149,999 2017', 'HHIncome:150 to 199,999 2017', 'HHIncome%:150 to 199,999 2017', 'HHIncome:200K or More 2017', 
#         'HHIncome%:200K or More 2017', 'Earnings:Less than 5,000 2017', 'Earnings:% Less than 5,000 2017', 'Earnings:5,000 to 9,999 2017', 
#         'Earnings:% 5,000 to 9,999 2017', 'Earnings:Less than 10,000 2017', 'Earnings:% Less than 10,000 2017', 'Earnings:10,000 to 19,999 2017', 
#         'Earnings:% 10,000 to 19,999 2017', 'Earnings:20,000 to 29,999 2017', 'Earnings:% 20,000 to 29,999 2017', 'Earnings:30,000 to 39,999 2017', 
#         'Earnings:% 30,000 to 39,999 2017', 'Earnings:40,000 to 49,999 2017', 'Earnings:% 40,000 to 49,999 2017', 'Earnings:50,000 to 64,999 2017', 
#         'Earnings:% 50,000 to 64,999 2017', 'Earnings:65,000 to 74,999 2017', 'Earnings:% 65,000 to 74,999 2017', 'Earnings:Less than 25,000 2017', 
#         'Earnings:% Less than 25,000 2017', 'Earnings:25,000 to 49,999 2017', 'Earnings:% 25,000 to 49,999 2017', 'Earnings:50,000 to 74,999 2017', 
#         'Earnings:% 50,000 to 74,999 2017', 'Earnings:75,000 to 99,999 2017', 'Earnings:% 75,000 to 99,999 2017', 'Earnings:100,000 and Over 2017', 
#         'Earnings:% 100,000 and Over 2017', 'Earnings:Median, All Earners 2017', 'EarningsFT:Less than 5,000 2017', 'EarningsFT:% Less than 5,000 2017', 
#         'EarningsFT:5,000 to 9,999 2017', 'EarningsFT:% 5,000 to 9,999 2017', 'EarningsFT:Less than 10,000 2017', 'EarningsFT:% Less than 10,000 2017', 
#         'EarningsFT:10,000 to 19,999 2017', 'EarningsFT:% 10,000 to 19,999 2017', 'EarningsFT:20,000 to 29,999 2017', 'EarningsFT:% 20,000 to 29,999 2017', 
#         'EarningsFT:30,000 to 39,999 2017', 'EarningsFT:% 30,000 to 39,999 2017', 'EarningsFT:40,000 to 49,999 2017', 'EarningsFT:% 40,000 to 49,999 2017', 
#         'EarningsFT:50,000 to 64,999 2017', 'EarningsFT:% 50,000 to 64,999 2017', 'EarningsFT:65,000 to 74,999 2017', 'EarningsFT:% 65,000 to 74,999 2017', 
#         'EarningsFT:Less than 25,000 2017', 'EarningsFT:% Less than 25,000 2017', 'EarningsFT:25,000 to 49,999 2017', 'EarningsFT:% 25,000 to 49,999 2017', 
#         'EarningsFT:50,000 to 74,999 2017', 'EarningsFT:% 50,000 to 74,999 2017', 'EarningsFT:75,000 to 99,999 2017', 'EarningsFT:% 75,000 to 99,999 2017', 
#         'EarningsFT:100,000 and Over 2017', 'EarningsFT:% 100,000 and Over 2017', 'Median Gross Rent 2017', 'Median Monthly Owner Costs 2017', 
#         'Median Monthly Owner Costs Home with Mortgage 2017', 'Median Monthly Owner Costs Home without Mortgage 2017', 
#         'Commute:Workers 16+ Not Working at Home 2017', 
#         'Commute:Less than 10 Minutes 2017', 'Commute%:Less than 10 Minutes 2017', 
#         'Commute:10 to 14 Minutes 2017', 'Commute%:10 to 14 Minutes 2017', 
#         'Commute:15 to 19 Minutes 2017', 'Commute%:15 to 19 Minutes 2017', 
#         'Commute:20 to 24 Minutes 2017', 'Commute%:20 to 24 Minutes 2017',
#         'Commute:25 to 29 Minutes 2017', 'Commute%:25 to 29 Minutes 2017',
#         'Commute:30 to 34 Minutes 2017', 'Commute%:30 to 34 Minutes 2017',
#         'Commute:35 to 44 Minutes 2017', 'Commute%:35 to 44 Minutes 2017', 
#         'Commute:45 to 59 Minutes 2017', 'Commute%:45 to 59 Minutes 2017',
        'Commute:60 or More Minutes 2017', 'Commute%:60 or More Minutes 2017',
        'Commute:45 or More Minutes 2017', 'Commute%:45 or More Minutes 2017', 
        
#         'Transpo:Workers Commuting 2017', 'Transpo:Car, Truck, or Van 2017', 
#         'Transpo%:Car, Truck, or Van 2017', 
        'Transpo:Car, Truck, or Van:Drove Alone 2017', 'Transpo%:Car, Truck, or Van:Drove Alone 2017', 
        'Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2017', 'Transpo:Car, Truck, or Van:Carpooled 2017', 'Transpo%:Car, Truck, or Van:Carpooled 2017', 
        'Transpo:Public Transportation 2017', 'Transpo%:Public Transportation 2017', 'Transpo:Bicycle 2017', 'Transpo%:Bicycle 2017', 'Transpo:Walk 2017', 
        'Transpo%:Walk 2017', 'Transpo:Worked From Home 2017', 'Transpo%:Worked From Home 2017', 'Transpo:Taxi, Motorcycle, Other 2017', 
        'Transpo%:Taxi, Motorcycle, Other 2017', 
        
        'Mean Travel Time to Work 2017', 
        
        'CB:Cost Burdened Renters 2017', 'CB%:Cost Burdened Renters 2017', 'CB:Cost Burdened Homeowners 2017', 'CB%: Cost Burdened Homeowners 2017', 
        'CB:Cost Burdened Households 2017', 'CB%:Cost Burdened Households 2017', 
        
        'Poverty%: Below Poverty Level 2017', 'Poverty: Population Below Poverty Level 2017',
        
        'Median Home Value for Owner Occupied Units 2017',
        
        
#         'Internet:Has a Computer 2022', 'Internet%:Has a Computer 2022', 'Internet:Has a Computer Under 18 2022', 'Internet%:Under 18 has a Computer 2022', 
#         'Internet:Has a Computer 18 to 64 2022', 'Internet%:18 to 64 has a Computer 2022', 'Internet:Has a Computer 65 Over 2022', 
#         'Internet%:65 Over has a Computer 2022', 'Internet:Has a Computer with Dial Up Only 2022', 'Internet%:Has a Computer with Dial Up Only 2022', 
#         'Internet:Has a Computer with Dial Up Only Under 18 2022', 'Internet%:Under 18 has a Computer with Dial Up Only 2022', 
#         'Internet:Has a Computer with Dial Up Only 18 to 64 2022', 'Internet%:18 to 64 has a Computer with Dial Up Only 2022', 
#         'Internet:Has a Computer with Dial Up Only 65 Over 2022', 'Internet%:65 Over has a Computer with Dial Up Only 2022', 
        'Internet:Has a Computer with Broadband 2022', 'Internet%:Has a Computer with Broadband 2022', 
#         'Internet:Has a Computer with Broadband Under 18 2022', 
#         'Internet%:Under 18 has a Computer with Broadband 2022', 'Internet:Has a Computer with Broadband 18 to 64 2022', 
#         'Internet%:18 to 64 has a Computer with Broadband 2022', 'Internet:Has a Computer with Broadband 65 Over 2022', 
#         'Internet%:65 Over has a Computer with Broadband 2022', 'Internet:Has a Computer with No Internet 2022', 'Internet%:Has a Computer with No Internet 2022', 
#         'Internet:Has a Computer with No Internet Under 18 2022', 'Internet%:Under 18 has a Computer with No Internet 2022', 
#         'Internet:Has a Computer with No Internet 18 to 64 2022', 'Internet%:18 to 64 has a Computer with No Internet 2022', 
#         'Internet:Has a Computer with No Internet 65 Over 2022', 'Internet%:65 Over has a Computer with No Internet 2022', 'Internet:Has No Computer 2022', 
#         'Internet%:Has No Computer 2022', 'Internet:Has No Computer Under 18 2022', 'Internet%:Under 18 has No Computer 2022', 
#         'Internet:Has No Computer 18 to 64 2022', 'Internet%:18 to 64 has No Computer 2022', 'Internet:Has No Computer 65 Over 2022', 
#         'Internet%:65 Over has No Computer 2022', 
        
#         'Ed:Population 25+ Educational Attainment 2022', 'Ed:Less than High School 2022', 'Ed%:Less than High School 2022', 
#         'Ed:High School Graduate or Equivalency 2022', 'Ed%:High School Graduate or Equivalency 2022', 
        'Ed:High School Graduate or More 2022', 'Ed%:High School Graduate or More 2022', 
#         'Ed:Some College 2022', 'Ed%:Some College 2022', 'Ed%:Some College or More 2022', 'Ed:Associates 2022', 
#         'Ed%:Associates 2022', 'Ed:Bachelors 2022', 'Ed%:Bachelors 2022', 
        'Ed:Bachelors or More 2022', 'Ed%:Bachelors or More 2022', 
#         'Ed:Masters 2022', 
#         'Ed:Masters or More 2022', 'Ed%:Masters or More 2022', 'Ed%:Masters 2022', 'Ed:Professional School Degree 2022', 'Ed%:Professional School Degree 2022', 
#         'Ed%:Professional School Degree or More 2022', 'Ed:Doctorate Degree 2022', 'Ed%:Doctorate Degree 2022', 
#         'Employment:Armed Forces 2022', 
#         'Employment:Not in Labor Force 2022', 'Employment:Employed Civilian 2022', 'Employment:Unemployed Civilian 2022', 
#         'Employment%:Civilian Population in LF 16 Over that are Employed 2022', 'Employment%:Total Civilian Population 16 Over that are Employed 2022', 
#         'Employment%:Population 16 Over in LF that are Employed 2022', 'Employment%:Population 16 Over that are Employed Civilians 2022', 
#         'Employment%:Civilian Population in LF 16 Over that are Unemployed 2022', 'Employment%:Total Civilian Population 16 Over that are Unemployed 2022', 
#         'Employment%:Population 16 Over in LF that are Unemployed 2022', 'Employment%:Population 16 Over that are Unemployed Civilians 2022', 
#         'Employment%:Civilian Population 16 Over Not in Labor Force 2022', 'Employment%:Total Population 16 Over Not in Labor Force 2022', 
#         'Employment:Total Population 16 Over In Labor Force 2022', 'Employment%:Total Population 16 Over In Labor Force 2022', 
        'Employment:Civilian Population 16 Over In Labor Force 2022', 'Employment%:Civilian Population 16 Over In Labor Force 2022', 
#         'Employment%:Population 16 Over in LF Employed in Armed Forces 2022', 'Employment%:Population 16 Over Employed in Armed Forces 2022', 
#         'HHIncome:Median for Householder Under 25 2022', 'HHIncome:Median for Householder Under 25 to 44 2022', 
#         'HHIncome:Median for Householder Under 45 to 64 2022', 'HHIncome:Median for Householder Under 65 Over 2022', 
        
        'Median Household Income 2022', 
        
#         'HHIncome:Total Households 2022', 'HHIncome:Less than 10,000 2022', 'HHIncome%:Less than 10,000 2022', 'HHIncome:10 to 14,999 2022', 
#         'HHIncome%:10 to 14,999 2022', 'HHIncome:15 to 19,999 2022', 'HHIncome%:15 to 19,999 2022', 'HHIncome:20 to 24,999 2022', 'HHIncome%:20 to 24,999 2022', 
#         'HHIncome:25 to 29,999 2022', 'HHIncome%:25 to 29,999 2022', 'HHIncome:30 to 34,999 2022', 'HHIncome:%30 to 34,999 2022', 'HHIncome:35 to 39,999 2022', 
#         'HHIncome%:35 to 39,999 2022', 'HHIncome:40 to 44,999 2022', 'HHIncome%:40 to 44,999 2022', 'HHIncome:45 to 49,999 2022', 'HHIncome%:45 to 49,999 2022', 
#         'HHIncome:50 to 59,999 2022', 'HHIncome%:50 to 59,999 2022', 'HHIncome:60 to 74,999 2022', 'HHIncome%:60 to 74,999 2022', 'HHIncome:75 to 99,999 2022', 
#         'HHIncome%:75 to 99,999 2022', 'HHIncome:100 to 124,999 2022', 'HHIncome%:100 to 124,999 2022', 'HHIncome:125 to 149,999 2022', 
#         'HHIncome%:125 to 149,999 2022', 'HHIncome:150 to 199,999 2022', 'HHIncome%:150 to 199,999 2022', 'HHIncome:200K or More 2022', 
#         'HHIncome%:200K or More 2022', 'Earnings:Less than 5,000 2022', 'Earnings:% Less than 5,000 2022', 'Earnings:5,000 to 9,999 2022', 
#         'Earnings:% 5,000 to 9,999 2022', 'Earnings:Less than 10,000 2022', 'Earnings:% Less than 10,000 2022', 'Earnings:10,000 to 19,999 2022', 
#         'Earnings:% 10,000 to 19,999 2022', 'Earnings:20,000 to 29,999 2022', 'Earnings:% 20,000 to 29,999 2022', 'Earnings:30,000 to 39,999 2022', 
#         'Earnings:% 30,000 to 39,999 2022', 'Earnings:40,000 to 49,999 2022', 'Earnings:% 40,000 to 49,999 2022', 'Earnings:50,000 to 64,999 2022', 
#         'Earnings:% 50,000 to 64,999 2022', 'Earnings:65,000 to 74,999 2022', 'Earnings:% 65,000 to 74,999 2022', 'Earnings:Less than 25,000 2022', 
#         'Earnings:% Less than 25,000 2022', 'Earnings:25,000 to 49,999 2022', 'Earnings:% 25,000 to 49,999 2022', 'Earnings:50,000 to 74,999 2022', 
#         'Earnings:% 50,000 to 74,999 2022', 'Earnings:75,000 to 99,999 2022', 'Earnings:% 75,000 to 99,999 2022', 'Earnings:100,000 and Over 2022', 
#         'Earnings:% 100,000 and Over 2022', 'Earnings:Median, All Earners 2022', 'EarningsFT:Less than 5,000 2022', 'EarningsFT:% Less than 5,000 2022', 
#         'EarningsFT:5,000 to 9,999 2022', 'EarningsFT:% 5,000 to 9,999 2022', 'EarningsFT:Less than 10,000 2022', 'EarningsFT:% Less than 10,000 2022', 
#         'EarningsFT:10,000 to 19,999 2022', 'EarningsFT:% 10,000 to 19,999 2022', 'EarningsFT:20,000 to 29,999 2022', 'EarningsFT:% 20,000 to 29,999 2022', 
#         'EarningsFT:30,000 to 39,999 2022', 'EarningsFT:% 30,000 to 39,999 2022', 'EarningsFT:40,000 to 49,999 2022', 'EarningsFT:% 40,000 to 49,999 2022', 
#         'EarningsFT:50,000 to 64,999 2022', 'EarningsFT:% 50,000 to 64,999 2022', 'EarningsFT:65,000 to 74,999 2022', 'EarningsFT:% 65,000 to 74,999 2022', 
#         'EarningsFT:Less than 25,000 2022', 'EarningsFT:% Less than 25,000 2022', 'EarningsFT:25,000 to 49,999 2022', 'EarningsFT:% 25,000 to 49,999 2022', 
#         'EarningsFT:50,000 to 74,999 2022', 'EarningsFT:% 50,000 to 74,999 2022', 'EarningsFT:75,000 to 99,999 2022', 'EarningsFT:% 75,000 to 99,999 2022', 
#         'EarningsFT:100,000 and Over 2022', 'EarningsFT:% 100,000 and Over 2022', 'Median Gross Rent 2022', 'Median Monthly Owner Costs 2022', 
#         'Median Monthly Owner Costs Home with Mortgage 2022', 'Median Monthly Owner Costs Home without Mortgage 2022', 
#         'Commute:Workers 16+ Not Working at Home 2022', 
#         'Commute:Less than 10 Minutes 2022', 'Commute%:Less than 10 Minutes 2022', 
#         'Commute:10 to 14 Minutes 2022', 'Commute%:10 to 14 Minutes 2022', 
#         'Commute:15 to 19 Minutes 2022', 'Commute%:15 to 19 Minutes 2022', 
#         'Commute:20 to 24 Minutes 2022', 'Commute%:20 to 24 Minutes 2022',
#         'Commute:25 to 29 Minutes 2022', 'Commute%:25 to 29 Minutes 2022',
#         'Commute:30 to 34 Minutes 2022', 'Commute%:30 to 34 Minutes 2022',
#         'Commute:35 to 44 Minutes 2022', 'Commute%:35 to 44 Minutes 2022', 
#         'Commute:45 to 59 Minutes 2022', 'Commute%:45 to 59 Minutes 2022',
        'Commute:60 or More Minutes 2022', 'Commute%:60 or More Minutes 2022',
        'Commute:45 or More Minutes 2022', 'Commute%:45 or More Minutes 2022',
#         'Transpo:Workers Commuting 2022', 'Transpo:Car, Truck, or Van 2022', 
#         'Transpo%:Car, Truck, or Van 2022', 
        'Transpo:Car, Truck, or Van:Drove Alone 2022', 'Transpo%:Car, Truck, or Van:Drove Alone 2022', 
        'Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2022', 'Transpo:Car, Truck, or Van:Carpooled 2022', 'Transpo%:Car, Truck, or Van:Carpooled 2022', 
        'Transpo:Public Transportation 2022', 'Transpo%:Public Transportation 2022', 'Transpo:Bicycle 2022', 'Transpo%:Bicycle 2022', 'Transpo:Walk 2022', 
        'Transpo%:Walk 2022', 'Transpo:Worked From Home 2022', 'Transpo%:Worked From Home 2022', 'Transpo:Taxi, Motorcycle, Other 2022', 
        'Transpo%:Taxi, Motorcycle, Other 2022',  
        
        'CB:Cost Burdened Renters 2022', 'CB%:Cost Burdened Renters 2022', 'CB:Cost Burdened Homeowners 2022', 'CB%: Cost Burdened Homeowners 2022', 
        'CB:Cost Burdened Households 2022', 'CB%:Cost Burdened Households 2022', 
        
        'Poverty%: Below Poverty Level 2022', 'Poverty: Population Below Poverty Level 2022', 
        
        'Median Home Value for Owner Occupied Units 2022', 
        
        'Mean Travel Time to Work 2022',

       ]
data = data[cols]
data.head()

,NAME,Internet:Has a Computer with Broadband 2017,Internet%:Has a Computer with Broadband 2017,Ed:High School Graduate or More 2017,Ed%:High School Graduate or More 2017,Ed:Bachelors or More 2017,Ed%:Bachelors or More 2017,Employment:Civilian Population 16 Over In Labor Force 2017,Employment%:Civilian Population 16 Over In Labor Force 2017,Median Household Income 2017,Commute:60 or More Minutes 2017,Commute%:60 or More Minutes 2017,Commute:45 or More Minutes 2017,Commute%:45 or More Minutes 2017,"Transpo:Car, Truck, or Van:Drove Alone 2017","Transpo%:Car, Truck, or Van:Drove Alone 2017","Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2017","Transpo:Car, Truck, or Van:Carpooled 2017","Transpo%:Car, Truck, or Van:Carpooled 2017",Transpo:Public Transportation 2017,Transpo%:Public Transportation 2017,Transpo:Bicycle 2017,Transpo%:Bicycle 2017,Transpo:Walk 2017,Transpo%:Walk 2017,Transpo:Worked From Home 2017,Transpo%:Worked From Home 2017,"Transpo:Taxi, Motorcycle, Other 2017","Transpo%:Taxi, Motorcycle, Other 2017",Mean Travel Time to Work 2017,CB:Cost Burdened Renters 2017,CB%:Cost Burdened Renters 2017,CB:Cost Burdened Homeowners 2017,CB%: Cost Burdened Homeowners 2017,CB:Cost Burdened Households 2017,CB%:Cost Burdened Households 2017,Poverty%: Below Poverty Level 2017,Poverty: Population Below Poverty Level 2017,Median Home Value for Owner Occupied Units 2017,Internet:Has a Computer with Broadband 2022,Internet%:Has a Computer with Broadband 2022,Ed:High School Graduate or More 2022,Ed%:High School Graduate or More 2022,Ed:Bachelors or More 2022,Ed%:Bachelors or More 2022,Employment:Civilian Population 16 Over In Labor Force 2022,Employment%:Civilian Population 16 Over In Labor Force 2022,Median Household Income 2022,Commute:60 or More Minutes 2022,Commute%:60 or More Minutes 2022,Commute:45 or More Minutes 2022,Commute%:45 or More Minutes 2022,"Transpo:Car, Truck, or Van:Drove Alone 2022","Transpo%:Car, Truck, or Van:Drove Alone 2022","Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2022","Transpo:Car, Truck, or Van:Carpooled 2022","Transpo%:Car, Truck, or Van:Carpooled 2022",Transpo:Public Transportation 2022,Transpo%:Public Transportation 2022,Transpo:Bicycle 2022,Transpo%:Bicycle 2022,Transpo:Walk 2022,Transpo%:Walk 2022,Transpo:Worked From Home 2022,Transpo%:Worked From Home 2022,"Transpo:Taxi, Motorcycle, Other 2022","Transpo%:Taxi, Motorcycle, Other 2022",CB:Cost Burdened Renters 2022,CB%:Cost Burdened Renters 2022,CB:Cost Burdened Homeowners 2022,CB%: Cost Burdened Homeowners 2022,CB:Cost Burdened Households 2022,CB%:Cost Burdened Households 2022,Poverty%: Below Poverty Level 2022,Poverty: Population Below Poverty Level 2022,Median Home Value for Owner Occupied Units 2022,Mean Travel Time to Work 2022
0,Cheatham County,31792.0,89.035763,23518.0,86.086606,5301.0,19.404078,20290.0,64.087176,56150.0,1823.0,10.140735,4327.0,24.069645,15790.0,83.962565,90.254358,1705.0,9.745642,66.0,0.350952,0.0,0.000000,185.0,0.983729,829.0,4.408168,231.0,1.228331,32.045391,1407.0,45.800781,2539.0,21.889818,3946.0,26.896599,12.651907,4966.0,170500.0,36355.0,93.409558,26097.0,88.946830,6717.0,22.893661,21614.0,64.799880,77014.0,2181.0,11.881026,5866.0,31.955112,15491.0,75.899069,86.459787,2426.0,13.540213,44.0,0.215581,0.0,0.000000,55.0,0.269476,2053.0,10.058795,341.0,1.670750,904.0,30.550862,2107.0,16.526786,3011.0,19.168577,9.342212,3802.0,262200.0,34.441358
1,Davidson County,534224.0,88.282955,408458.0,88.024809,181425.0,39.098025,384413.0,70.376181,53419.0,15727.0,4.686471,36504.0,10.877786,281729.0,79.204327,89.012215,34777.0,10.987785,7621.0,2.142542,726.0,0.204105,7267.0,2.043020,20116.0,5.655343,3463.0,0.973576,24.525348,56630.0,45.388968,34570.0,23.243305,91200.0,33.345887,16.892091,111103.0,194800.0,634842.0,95.083057,447505.0,90.335520,227800.0,45.984808,415160.0,71.689688,71863.0,16098.0,4.879986,38660.0,11.719484,276136.0,70.703305,89.393910,32762.0,10.606090,6034.0,1.544977,1132.0,0.289843,8114.0,2.077551,60

In [77]:
# test = data[['NAME', 'Commute:Workers 16+ Not Working at Home 2021']]
# test.to_csv('commuters.csv')

In [78]:
data.set_index('NAME', inplace = True)
cols = data.columns
data[cols] = data[cols].astype(float)

In [79]:
for column in data.columns:
    if '%' in column:
        data[column] /= 100

In [80]:
data.head()

,Internet:Has a Computer with Broadband 2017,Internet%:Has a Computer with Broadband 2017,Ed:High School Graduate or More 2017,Ed%:High School Graduate or More 2017,Ed:Bachelors or More 2017,Ed%:Bachelors or More 2017,Employment:Civilian Population 16 Over In Labor Force 2017,Employment%:Civilian Population 16 Over In Labor Force 2017,Median Household Income 2017,Commute:60 or More Minutes 2017,Commute%:60 or More Minutes 2017,Commute:45 or More Minutes 2017,Commute%:45 or More Minutes 2017,"Transpo:Car, Truck, or Van:Drove Alone 2017","Transpo%:Car, Truck, or Van:Drove Alone 2017","Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2017","Transpo:Car, Truck, or Van:Carpooled 2017","Transpo%:Car, Truck, or Van:Carpooled 2017",Transpo:Public Transportation 2017,Transpo%:Public Transportation 2017,Transpo:Bicycle 2017,Transpo%:Bicycle 2017,Transpo:Walk 2017,Transpo%:Walk 2017,Transpo:Worked From Home 2017,Transpo%:Worked From Home 2017,"Transpo:Taxi, Motorcycle, Other 2017","Transpo%:Taxi, Motorcycle, Other 2017",Mean Travel Time to Work 2017,CB:Cost Burdened Renters 2017,CB%:Cost Burdened Renters 2017,CB:Cost Burdened Homeowners 2017,CB%: Cost Burdened Homeowners 2017,CB:Cost Burdened Households 2017,CB%:Cost Burdened Households 2017,Poverty%: Below Poverty Level 2017,Poverty: Population Below Poverty Level 2017,Median Home Value for Owner Occupied Units 2017,Internet:Has a Computer with Broadband 2022,Internet%:Has a Computer with Broadband 2022,Ed:High School Graduate or More 2022,Ed%:High School Graduate or More 2022,Ed:Bachelors or More 2022,Ed%:Bachelors or More 2022,Employment:Civilian Population 16 Over In Labor Force 2022,Employment%:Civilian Population 16 Over In Labor Force 2022,Median Household Income 2022,Commute:60 or More Minutes 2022,Commute%:60 or More Minutes 2022,Commute:45 or More Minutes 2022,Commute%:45 or More Minutes 2022,"Transpo:Car, Truck, or Van:Drove Alone 2022","Transpo%:Car, Truck, or Van:Drove Alone 2022","Transpo%:Car, Truck, or Van:Drove Alone Share of Drivers 2022","Transpo:Car, Truck, or Van:Carpooled 2022","Transpo%:Car, Truck, or Van:Carpooled 2022",Transpo:Public Transportation 2022,Transpo%:Public Transportation 2022,Transpo:Bicycle 2022,Transpo%:Bicycle 2022,Transpo:Walk 2022,Transpo%:Walk 2022,Transpo:Worked From Home 2022,Transpo%:Worked From Home 2022,"Transpo:Taxi, Motorcycle, Other 2022","Transpo%:Taxi, Motorcycle, Other 2022",CB:Cost Burdened Renters 2022,CB%:Cost Burdened Renters 2022,CB:Cost Burdened Homeowners 2022,CB%: Cost Burdened Homeowners 2022,CB:Cost Burdened Households 2022,CB%:Cost Burdened Households 2022,Poverty%: Below Poverty Level 2022,Poverty: Population Below Poverty Level 2022,Median Home Value for Owner Occupied Units 2022,Mean Travel Time to Work 2022
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cheatham County,31792.0,0.890358,23518.0,0.860866,5301.0,0.194041,20290.0,0.640872,56150.0,1823.0,0.101407,4327.0,0.240696,15790.0,0.839626,0.902544,1705.0,0.097456,66.0,0.003510,0.0,0.000000,185.0,0.009837,829.0,0.044082,231.0,0.012283,32.045391,1407.0,0.458008,2539.0,0.218898,3946.0,0.268966,0.126519,4966.0,170500.0,36355.0,0.934096,26097.0,0.889468,6717.0,0.228937,21614.0,0.647999,77014.0,2181.0,0.118810,5866.0,0.319551,15491.0,0.758991,0.864598,2426.0,0.135402,44.0,0.002156,0.0,0.000000,55.0,0.002695,2053.0,0.100588,341.0,0.016707,904.0,0.305509,2107.0,0.165268,3011.0,0.191686,0.093422,3802.0,262200.0,34.441358
Davidson County,534224.0,0.882830,408458.0,0.880248,181425.0,0.390980,384413.0,0.703762,53419.0,15727.0,0.046865,36504.0,0.108778,281729.0,0.792043,0.890122,34777.0,0.109878,7621.0,0.021425,726.0,0.002041,7267.0,0.020430,20116.0,0.056553,3463.0,0.009736,24.525348,56630.0,0.453890,34570.0,0.232433,91200.0,0.333459,0.168921,111103.0,194800.0,634842.0,0.950831,447505.0,0.903355,227800.0,0.459848,415160.0,0.716897,71863.0,16098.0,0.048800,38660.0,0.117195,276136.0,0.707033,0.893939,32762.0,0.106061,6034.0,0.015450,1132.0,

In [81]:
data.to_csv('../data/CEDS_2023Update_CensusBureauData.csv')